In [1]:

import sys

sys.path.append("../")
from pathlib import Path
import polars as pl
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

from common.make_vec_env import make_vec_env
from envs.single_stock_trading_past_n_price_portfolio_reward_env import StockTradingEnv

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
TICKER = "SBIN.NS"
TRAIN_FILE = Path("../datasets") / f"{TICKER}_train"
EVAL_FILE = Path("datasets") / f"{TICKER}_trade"

In [4]:
CLOSE_PRICES = (
    pl
    .read_parquet(TRAIN_FILE)
    .with_columns(index=pl.int_range(0, end=pl.count()))
    .sort("index")
    .set_sorted("index")
    .group_by_dynamic(
        "index", every="1i", period="40i", include_boundaries=True, closed="right"
    )
    .agg(pl.col("Close"))
    .with_columns(pl.col("Close").list.len().alias("Count"))
    .filter(pl.col("Count") == 40)["Close"]
    .to_numpy()
)


In [5]:
check_env(StockTradingEnv(CLOSE_PRICES, seed=0))

In [79]:
from envs.single_stock_trading_past_n_price_portfolio_reward_env import StockTradingEnv
env = StockTradingEnv(CLOSE_PRICES, seed=0)
obs, info = env.reset()
obs, info

(array([  457.65,   458.  ,   448.95,   447.4 ,   446.9 ,   447.4 ,
          447.2 ,   446.9 ,   449.2 ,   460.  ,   459.4 ,   458.55,
          463.45,   462.9 ,   462.8 ,   462.4 ,   465.75,   466.65,
          470.45,   467.75,   464.  ,   460.55,   461.  ,   464.9 ,
          462.95,   465.35,   462.2 ,   462.45,   461.65,   463.4 ,
          461.55,   463.5 ,   458.65,   457.1 ,   458.2 ,   454.8 ,
          454.1 ,   454.  ,   453.4 ,   453.35, 10000.  ,     0.  ,
            0.  ,     0.  ,     0.  , 10000.  , 10000.  ], dtype=float32),
 {})

In [164]:
#{0: "HOLD", 1: "BUY", 2: "SELL"}
s, r, d, t, i = env.step(0)
i['counter'], i['close_price'], i['buy_price'], i['predicted_action'], r

(83, 446.2, 453.35, 'HOLD', -157.2998046875)

In [71]:
470.2 * 22 + 26

10370.4

In [114]:
i

{'seed': 0,
 'counter': 33,
 'close_price': 471.45,
 'predicted_action': 'HOLD',
 'description': '[+] Transaction #33: Profitable Holding. Shares Held: 22.0. Purchase Price per Share: ₹453.35. Current Share Price: ₹471.45. Profit/Loss: ₹398.2001953125 Holding Performance: Good Streak - 33, Bad Streak - 0. Reward Earned: ₹398.2001953125. Portfolio Value: ₹10398.2001953125.Portfolio Value Threshold: ₹10000.0.',
 'available_amount': 26.299805,
 'shares_holdings': 22.0,
 'buy_price': 453.35,
 'buy_price_index': 0,
 'reward': 398.2001953125,
 'done': False,
 'truncated': False,
 'correct_trade': 1,
 'wrong_trade': 0,
 'correct_trade %': 100.0,
 'buy_counter': 1,
 'sell_counter': 0,
 'hold_counter': 33,
 'good_hold_counter': 33,
 'good_sell_counter': 0,
 'good_buy_counter': 0,
 'bad_hold_counter': 0,
 'bad_sell_counter': 0,
 'bad_buy_counter': 1,
 'hold_with_no_shares_counter': 0,
 'bad_holds_with_no_shares_counter': 0,
 'good_holds_with_no_shares_counter': 0,
 'good_hold_streak': 33,
 'bad_

In [53]:
thresh = 20
def simplified_reward(percentage_distance):
    if percentage_distance <= thresh:
        return 100 - (percentage_distance / thresh) * 100
    else:
        return -((percentage_distance - thresh) / thresh) * 100
    

simplified_reward(15)

25.0